# Preprocessing raw scraped data

## Load data

In [146]:
import pandas as pd

path = '../data/raw/2024-01-01.csv'
df = pd.read_csv(path)

df.columns = 'hour', 'price', 'volume'
df

hour  price   volume
0   00h-01h  63.33   8744.9
1   01h-02h  50.09   8671.4
..      ...    ...      ...
22  22h-23h  47.50  11154.9
23  23h-24h  42.09  11000.1

[24 rows x 3 columns]

## Preprocessing

In [147]:
date = path.split('/')[-1].split('.')[0]
df['date'] = date

hour = df['hour'].str.extract(r'^(\d+)')[0]
df['hour'] = hour

s = df['date'] + ' ' + df['hour']
s = pd.to_datetime(s)

df['datetime'] = s
df.drop(columns=['date', 'hour'], inplace=True)

df

price   volume            datetime
0   63.33   8744.9 2024-01-01 00:00:00
1   50.09   8671.4 2024-01-01 01:00:00
..    ...      ...                 ...
22  47.50  11154.9 2024-01-01 22:00:00
23  42.09  11000.1 2024-01-01 23:00:00

[24 rows x 3 columns]

## Refactor into function

In [148]:
def preprocess(path):
    
    df = pd.read_csv(path)
    df.columns = 'hour', 'price', 'volume'
    
    date = path.split('/')[-1].split('.')[0]
    df['date'] = date
    
    hour = df['hour'].str.extract(r'^(\d+)')[0]
    df['hour'] = hour
    
    s = df['date'] + ' ' + df['hour']
    s = pd.to_datetime(s)
    df['datetime'] = s
    
    df.drop(columns=['date', 'hour'], inplace=True)
    
    return df

In [149]:
preprocess('../data/raw/2024-01-01.csv')

price   volume            datetime
0   63.33   8744.9 2024-01-01 00:00:00
1   50.09   8671.4 2024-01-01 01:00:00
..    ...      ...                 ...
22  47.50  11154.9 2024-01-01 22:00:00
23  42.09  11000.1 2024-01-01 23:00:00

[24 rows x 3 columns]

## Combine multiple files

### Replicate preprocessing

In [150]:
import glob
paths = glob.glob('../data/raw/*.csv')
paths

['../data/raw/2024-01-13.csv',
 '../data/raw/2024-01-07.csv',
 '../data/raw/2024-01-06.csv',
 '../data/raw/2024-01-12.csv',
 '../data/raw/2024-01-04.csv',
 '../data/raw/2024-01-10.csv',
 '../data/raw/2024-01-11.csv',
 '../data/raw/2024-01-05.csv',
 '../data/raw/2024-01-01.csv',
 '../data/raw/2024-01-15.csv',
 '../data/raw/2024-01-29.csv',
 '../data/raw/2024-01-28.csv',
 '../data/raw/2024-01-14.csv',
 '../data/raw/2024-01-16.csv',
 '../data/raw/2024-01-02.csv',
 '../data/raw/2024-04-27.csv',
 '../data/raw/2024-01-03.csv',
 '../data/raw/2024-01-17.csv',
 '../data/raw/2024-05-07.csv',
 '../data/raw/2024-01-26.csv',
 '../data/raw/2024-01-27.csv',
 '../data/raw/2024-01-19.csv',
 '../data/raw/2024-01-25.csv',
 '../data/raw/2024-01-31.csv',
 '../data/raw/2024-01-30.csv',
 '../data/raw/2024-01-24.csv',
 '../data/raw/2024-01-18.csv',
 '../data/raw/2024-01-20.csv',
 '../data/raw/2024-01-08.csv',
 '../data/raw/2024-01-09.csv',
 '../data/raw/2024-01-21.csv',
 '../data/raw/2024-01-23.csv',
 '../dat

In [151]:
dfs = []
for path in paths:
    df = preprocess(path)
    dfs.append(df)

### Concatenate `DataFrames`

In [152]:
df = pd.concat(dfs).set_index('datetime').sort_index()
df

price   volume
datetime                           
2024-01-01 00:00:00  63.33   8744.9
2024-01-01 01:00:00  50.09   8671.4
...                    ...      ...
2024-05-07 22:00:00  62.00  18313.9
2024-05-07 23:00:00  50.01  20672.7

[792 rows x 2 columns]

## Create new columns

In [153]:
df['cost'] = df['price'] * df['volume']
df

price   volume         cost
datetime                                        
2024-01-01 00:00:00  63.33   8744.9   553814.517
2024-01-01 01:00:00  50.09   8671.4   434350.426
...                    ...      ...          ...
2024-05-07 22:00:00  62.00  18313.9  1135461.800
2024-05-07 23:00:00  50.01  20672.7  1033841.727

[792 rows x 3 columns]

## Filter dates

In [155]:
df = df.loc['2024-01'].copy()
df

price   volume         cost
datetime                                        
2024-01-01 00:00:00  63.33   8744.9   553814.517
2024-01-01 01:00:00  50.09   8671.4   434350.426
...                    ...      ...          ...
2024-01-31 22:00:00  85.00  12171.6  1034586.000
2024-01-31 23:00:00  77.52  10826.8   839293.536

[744 rows x 3 columns]

## Export to Excel

In [157]:
df.to_excel('../data/processed/2024-01.xlsx')